# 必要なライブラリをインポート

In [1]:
# サンプルデータセットが用意されているライブラリ sklearn.datasets から
# ボストンの住宅価格データを取得するためのメソッド load_boston をインポート
from sklearn.datasets import load_boston

# pandas のインポート
import pandas as pd

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ  から
# モデル構築（訓練用）/検証データ分割用メソッド train_test_split をインポート
from sklearn.model_selection import train_test_split

# 機械学習用ライブラリ sklearn（scikit-learn）から線形回帰用クラス linear_model 中の
# Ridge回帰用クラス Ridge をインポート 
from sklearn.linear_model import Ridge

# 機械学習用ライブラリ sklearn（scikit-learn）内にあるライブラリ preprocessing から
# 標準化用クラス StandardScaler をインポート 
from sklearn.preprocessing import StandardScaler

# 統計解析用ライブラリ statsmodels 内にあるライブラリ stats.outliers_influence から
# 分散拡大係数（VIF）計算用メソッド variance_inflation_factor をインポート
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 精度評価指標を計算するためのメソッドをインポート
#   ・r2_score：決定係数
#   ・mean_squared_error：平均二乗誤差
#   ・mean_absolute_error：平均絶対誤差
#   ・median_absolute_error：Median Absolute Error
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error

# グラフ描画用ライブラリ matplotlib、seaborn をインポート
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 分散拡大係数（VIF）を確認するための関数を定義しておく

In [2]:
# 投入したデータセットの全ての変数についてVIFを計算する関数 checkVIF の定義
def checkVIF( ExplanatoryVarDataSet ):
  tmp_columnList = ExplanatoryVarDataSet.columns
  vifList = []
  for i in range(len(tmp_columnList)):
    colname = tmp_columnList[i]
    vif = variance_inflation_factor(ExplanatoryVarDataSet.values, i)
    vifList.append( [ colname, vif ] )
  return  pd.DataFrame( vifList, columns=["COLUMN","VIF"] )

# データの読み込み

In [3]:
# ボストン住宅価格データを読み込む
loadBoston = load_boston()
boston = pd.DataFrame(loadBoston.data, columns = loadBoston.feature_names)
boston["MEDV"] = loadBoston.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# 全データをモデル構築用データ、モデル検証用データに分割する

In [4]:
# 目的変数と説明変数に分割
columnList = boston.columns.values.tolist()
columnList.remove("MEDV")
X = boston.loc[:,columnList]
y = boston.loc[:,["MEDV"]]

# モデル構築用データ、モデル検証用データに分割（70:30に分割）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


# 標準化の実施

In [5]:
# X_train のデータを使い標準化パラメータを獲得してから、X_train、X_test に対して標準化を実施
scaler_X = StandardScaler()
scaler_X.fit( X_train )
X_train_std = pd.DataFrame(scaler_X.transform(X_train), columns=columnList)
X_test_std = pd.DataFrame(scaler_X.transform(X_test), columns=columnList)

In [6]:
# y_train のデータを使い標準化パラメータを獲得してから、y_train、y_test に対して標準化を実施
scaler_y = StandardScaler()
scaler_y.fit( y_train )
y_train_std = pd.DataFrame(scaler_y.transform(y_train), columns=["MEDV"])
y_test_std = pd.DataFrame(scaler_y.transform(y_test), columns=["MEDV"])

# Ridge回帰分析を実行する（正則化パラメータ：α=1）
* α=10のときと偏回帰係数の大きさが異なっている事に注目（10よりも少し大きくなっている、すなわちα=10のときより、正則化の効果が弱くなっている。）

In [7]:
# モデルのインスタンスを生成
reg = Ridge(alpha=1,random_state=1234)

# 回帰を実行する
reg.fit(X_train_std, y_train_std)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=1234, solver='auto', tol=0.001)

In [8]:
# 分析結果として、回帰係数（reg.coef_）、切片（reg.intercept_）を表示する
print(reg.coef_)
print(reg.intercept_[0])

[[-0.11689563  0.08407187  0.03350698  0.08740323 -0.18789675  0.31144826
  -0.03217017 -0.3095735   0.21468959 -0.14678067 -0.21771141  0.1088992
  -0.41290993]]
-1.3402802721230328e-16


In [9]:
# 多重共線性を確認する
vif = checkVIF(X_train_std[columnList])
vif["COEF"] = reg.coef_[0]
vif

,COLUMN,VIF,COEF
0,CRIM,1.731586,-0.116896
1,ZN,2.394452,0.084072
2,INDUS,3.836487,0.033507
3,CHAS,1.102642,0.087403
4,NOX,4.627219,-0.187897
5,RM,1.913056,0.311448
6,AGE,2.996110,-0.032170
7,DIS,3.971023,-0.309573
8,RAD,7.534681,0.214690
9,TAX,8.879471,-0.146781


In [10]:
# モデル構築用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_train_std, reg.predict(X_train_std)) ) )
print("MSE:" + str(mean_squared_error(y_train_std, reg.predict(X_train_std))) )
print("MAE:" + str(mean_absolute_error(y_train_std, reg.predict(X_train_std))) )
print("MedianAE:" + str(median_absolute_error(y_train_std, reg.predict(X_train_std))) )

R2 SCORE:0.7434823914140181
MSE:0.256517608585982
MAE:0.35754149684542325
MedianAE:0.2692535473792296


In [11]:
# モデル検証用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_test_std, reg.predict(X_test_std))) )
print("MSE:" + str(mean_squared_error(y_test_std, reg.predict(X_test_std))) )
print("MAE:" + str(mean_absolute_error(y_test_std, reg.predict(X_test_std))) )
print("MedianAE:" + str(median_absolute_error(y_test_std, reg.predict(X_test_std))) )

R2 SCORE:0.7108071617209415
MSE:0.2451593224440288
MAE:0.3371648367356463
MedianAE:0.2502121396411606


# Ridge回帰分析を実行する（正則化パラメータ：α=10）
* α=1のときと偏回帰係数の大きさが異なっている事に注目（少し小さくなっている）

In [12]:
# モデルのインスタンスを生成
reg = Ridge(alpha=10,random_state=1234)

# 回帰を実行する
reg.fit(X_train_std, y_train_std)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=1234, solver='auto', tol=0.001)

In [13]:
# 分析結果として、回帰係数（reg.coef_）、切片（reg.intercept_）を表示する
print(reg.coef_)
print(reg.intercept_[0])

[[-0.10758371  0.06972259  0.0138785   0.09126465 -0.15935183  0.31505123
  -0.03360185 -0.27660503  0.15921506 -0.10172641 -0.20855315  0.10645738
  -0.39993668]]
-9.833821740430116e-17


In [14]:
# 多重共線性を確認する
vif = checkVIF(X_train_std[columnList])
vif["COEF"] = reg.coef_[0]
vif

,COLUMN,VIF,COEF
0,CRIM,1.731586,-0.107584
1,ZN,2.394452,0.069723
2,INDUS,3.836487,0.013879
3,CHAS,1.102642,0.091265
4,NOX,4.627219,-0.159352
5,RM,1.913056,0.315051
6,AGE,2.996110,-0.033602
7,DIS,3.971023,-0.276605
8,RAD,7.534681,0.159215
9,TAX,8.879471,-0.101726


In [15]:
# モデル構築用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_train_std, reg.predict(X_train_std)) ) )
print("MSE:" + str(mean_squared_error(y_train_std, reg.predict(X_train_std))) )
print("MAE:" + str(mean_absolute_error(y_train_std, reg.predict(X_train_std))) )
print("MedianAE:" + str(median_absolute_error(y_train_std, reg.predict(X_train_std))) )

R2 SCORE:0.7423666687004578
MSE:0.25763333129954225
MAE:0.3549739457018454
MedianAE:0.2629949719539696


In [16]:
# モデル検証用データについて各精度評価指標を計算する
print("R2 SCORE:" + str(r2_score(y_test_std, reg.predict(X_test_std))) )
print("MSE:" + str(mean_squared_error(y_test_std, reg.predict(X_test_std))) )
print("MAE:" + str(mean_absolute_error(y_test_std, reg.predict(X_test_std))) )
print("MedianAE:" + str(median_absolute_error(y_test_std, reg.predict(X_test_std))) )

R2 SCORE:0.7072830902371284
MSE:0.2481468064438306
MAE:0.3361780351068509
MedianAE:0.25086326528427966
